In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import optuna
import optuna.visualization as vis
    
n = 9
    
X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):

    ### ========== Shuffle and Split Data ========== ###

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=42)  # 60% training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)  # 20% validation, 20% test

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    x_test_scaled = scaler.transform(X_test)


    ### ========== Train and Evaluate Model ========== ###

    classifier = RandomForestClassifier(
        n_estimators=100, 
        max_depth=trial.suggest_int('max_depth', 20, 50),
        max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        criterion=trial.suggest_categorical('criterion', ['entropy', 'log_loss']),
        n_jobs=-1
    )

    print(classifier)
    classifier.fit(X_train_scaled, y_train)
    # print("Fit Model")

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    # print(f"Validation Accuracy: {val_accuracy:.4f}")

    # Evaluate on the test set
    y_test_pred = classifier.predict(x_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    # print(f"Test Accuracy: {test_accuracy:.4f}")

    return val_accuracy


# Create a study object
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

vis.plot_optimization_history(study)
vis.plot_param_importances(study)
vis.plot_slice(study)

[I 2024-11-07 19:19:49,273] A new study created in memory with name: no-name-b4508d6f-2b99-4934-81d7-f393a2ee8130


RandomForestClassifier(criterion='log_loss', max_depth=27, max_features='log2',
                       n_jobs=-1)


[I 2024-11-07 19:19:50,152] Trial 0 finished with value: 0.5042592592592593 and parameters: {'max_depth': 27, 'max_features': 'log2', 'criterion': 'log_loss'}. Best is trial 0 with value: 0.5042592592592593.


RandomForestClassifier(criterion='log_loss', max_depth=33, max_features='log2',
                       n_jobs=-1)


[I 2024-11-07 19:19:51,045] Trial 1 finished with value: 0.5057407407407407 and parameters: {'max_depth': 33, 'max_features': 'log2', 'criterion': 'log_loss'}. Best is trial 0 with value: 0.5042592592592593.


...

RandomForestClassifier(criterion='entropy', max_depth=26, max_features='log2',
                       n_jobs=-1)


[I 2024-11-07 19:21:14,307] Trial 98 finished with value: 0.5027777777777778 and parameters: {'max_depth': 26, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 63 with value: 0.49574074074074076.


RandomForestClassifier(criterion='entropy', max_depth=29, max_features='log2',
                       n_jobs=-1)


[I 2024-11-07 19:21:15,162] Trial 99 finished with value: 0.5012962962962964 and parameters: {'max_depth': 29, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 63 with value: 0.49574074074074076.


Best hyperparameters: {'max_depth': 35, 'max_features': 'log2', 'criterion': 'entropy'}
Best value: 0.49574074074074076
